In [1]:
import random
import gym
import numpy as np
import random
import pandas as pd
import torch
import torch.nn as nn

import torch.optim as optim
from collections import deque
from sklearn.preprocessing import StandardScaler
import random
# Normalize features
from sklearn.preprocessing import StandardScaler
from collections import deque
import torch.nn.functional as F
import random
import gym
import numpy as np
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/kaggle/input/eqqerff/preprocessed.csv')
data.drop(columns= ['Unnamed: 0','index'], inplace = True)
# preprocessing
data['DATETIME'] = pd.to_datetime(data['DATETIME'])
data['TIME'] = data['TIME'].astype(str).str.rstrip('000')
data['hour'] = data['DATETIME'].dt.hour
data['minute'] = data['DATETIME'].dt.minute
data['day_of_week'] = data['DATETIME'].dt.day_name()
data['second'] =  data['DATETIME'].dt.second
day_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
data['day_of_week'] = data['day_of_week'].map(day_map)
data.drop(columns= ['DATETIME','TIME'], inplace = True) 
data.drop(columns= ['TRADE_PRICE','TRADENO'], inplace = True)
data= data.dropna()
data.drop(columns = ['DATE', 'ACTION','ORDERNO','TRADE_TIME'], inplace = True)
# Create a dictionary to map string values to numeric values
buysell_map = {'B': 0, 'S': 1}

# Map the string values to numeric values
data['BUYSELL'] = data['BUYSELL'].map(buysell_map)
data.drop(columns = 'TRADEPRICE', inplace = True)

In [3]:
from tqdm import tqdm
# Normalizing the data
def normalize_data(df):
    df_normalized = df.copy()
    for column in df.columns:
        if df[column].dtype == np.float64 or df[column].dtype == np.int64:
            df_normalized[column] = (df[column] - df[column].mean()) / df[column].std()
    return df_normalized

data = normalize_data(data)

In [4]:
#def reward_function(df):
 #   # Calculate best bid-ask spread for each date
  #  best_spread_per_date = df.groupby('DATE').apply(lambda x: x['BID_PRICE'].max() - x['ASK_PRICE'].min())

    #rewards = []
    #for index, row in df.iterrows():
        # Calculate execution price deviation from best bid-ask spread
       # execution_price_deviation = row['PRICE'] - best_spread_per_date[row['DATE']]

        # Calculate execution speed penalty
       # execution_speed_penalty = 100 / 1000  # converting milliseconds to seconds

      #  # Calculate commission cost
       # commission_cost = 0.01 * row['TRADE_VOLUME']
       # # Calculate market impact penalty (quadratic function)
       # market_impact_penalty = 0.0001 * (row['TRADE_VOLUME'] ** 2)

        # Calculate total reward
        #reward = -execution_price_deviation - execution_speed_penalty -  commission_cost- market_impact_penalty
        #rewards.append(reward)

   # return rewards
#
# rewards = reward_function(data)

look at how the prices move as volumes are traded

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from copy import deepcopy

# Define the Trading Environment
class TradingEnv:
    def __init__(self, data, ac_dict=None, ac_type='vanilla6'):
        self.data = data
        self.current_step = 0
        self.done = False
        
        # Define action space: 0 for hold, 1 for buy
        self.action_space = [0, 1]
        
        # Define observation space: based on the number of columns in data
        self.observation_space = len(data.columns)
        
        # Initialize other variables specific to your environment
        self.inventory = 0  # Starting with zero inventory
        
        # Metrics tracking
        self.cumulative_reward = 0
        self.cumulative_rewards = []
        
        # Assign action dictionary and type
        self.ac_dict = ac_dict if ac_dict is not None else {0: 'hold', 1: 'buy'}
        self.ac_type = ac_type
    
    def reset(self):
        self.current_step = 0
        self.done = False
        self.inventory = 0
        self.cumulative_reward = 0
        self.cumulative_rewards = []
        return self._next_observation()
    
    def _next_observation(self):
        # Return current observation based on the data at current_step
        obs = self.data.iloc[self.current_step].values
        return obs
    
    def step(self, action):
        # Take a step in the environment based on the action
        current_row = self.data.iloc[self.current_step]
        self.current_step += 1
        
        reward = self._calculate_reward(current_row, action)
        self.cumulative_reward += reward
        self.cumulative_rewards.append(self.cumulative_reward)
        
        # Update inventory based on action
        if action == 1:  # Buy action
            self.inventory += 1
        
        # Check if episode is done
        if self.current_step >= len(self.data) - 1:
            self.done = True
        
        next_observation = self._next_observation()
        return next_observation, reward, self.done
    
    def _calculate_reward(self, row, action):
        # Calculate reward based on action and current state (row)
        if action == 1:  # Buy action
            reward = row['PRICE']  # Example: reward could be the current price
        else:  # Hold action
            reward = 0  # No reward for holding
        
        return reward
    
    def render(self):
        # Optional: Implement rendering if needed for visualization
        pass
    
    def plot_cumulative_rewards(self):
        fig = px.line(
            x=range(len(self.cumulative_rewards)),
            y=self.cumulative_rewards,
            labels={'x': 'Steps', 'y': 'Cumulative Reward'},
            title='Cumulative Reward over Episodes'
        )
        fig.show()
        
    def calculate_parameters(self):
        # Extract necessary columns
        prices = self.data['PRICE']
        volumes = self.data['VOLUME']
        
        # Calculate sigma (volatility)
        returns = prices.pct_change().dropna()
        sigma = returns.std()

        # Calculate tau (average time between trades in seconds)
        time_diffs = self.data['second'].diff().dropna()
        tau = time_diffs.mean()

        # Calculate eta and rho based on a more detailed market impact model
        eta = self.calculate_eta()
        rho = self.calculate_rho()

        # Set a default value for lambda (risk aversion parameter)
        lamb = 1e-6

        # Calculate kappa
        kappa = np.sqrt(abs(lamb * sigma**2 / (eta * (1 - rho * tau / (2 * eta)))))
        
        return eta, rho, sigma, tau, lamb, kappa

    def calculate_eta(self):
        # Estimate the market impact cost per unit traded
        price_impact = self.data['PRICE'] - self.data['MID_PRICE']
        volume = self.data['VOLUME']
        
        # Filter out zero volumes to avoid division by zero
        valid_indices = volume > 0
        price_impact = price_impact[valid_indices]
        volume = volume[valid_indices]

        # Calculate the market impact per unit volume traded
        market_impact_per_volume = price_impact / volume
        eta = market_impact_per_volume.mean()  # Average market impact per unit volume

        return eta

    def calculate_rho(self):
        # Calculate the standard deviation of returns as a proxy for risk
        returns = self.data['PRICE'].pct_change().dropna()
        return_std = returns.std()

        # Risk aversion might be proportional to return standard deviation
        rho = return_std * 1e-2  # Adjust the scaling factor as necessary

        return rho

# Define the Almgren-Chriss Agent
class AlmgrenChrissAgent:
    def __init__(self, env, time_horizon, eta, rho, sigma, tau, lamb, kappa):
        self.ac_dict = deepcopy(env.ac_dict)
        self.ac_type = deepcopy(env.ac_type)
        self.eta = eta
        self.rho = rho
        self.sigma = sigma
        self.tau = tau
        self.time_horizon = time_horizon
        self.steps, self.j = time_horizon / self.tau, 1
        self.kappa = kappa

    def reset(self):
        self.j = 1

    def act(self, state):
        def closest_action(nj):
            # Since there are only two actions, use a simple threshold
            action = 1 if nj >= 0.5 else 0
            return action

        if self.ac_type in ['vanilla6', 'vanilla20']:
            inventory = state[1]  # Adjust index if necessary
            if self.kappa == 0:
                nj = self.tau / self.time_horizon * (1 / inventory)
            else:
                nj = 2 * np.sinh(0.5 * self.kappa * self.tau) * np.cosh(self.kappa * (
                        self.time_horizon - (self.j - 0.5) * self.tau)) * (1 / inventory) / np.sinh(self.kappa
                                                                                                    * self.time_horizon)
            self.j += 1
            if self.j == self.steps + 1:
                nj = 1
            action = closest_action(nj)

            return action
        elif self.ac_type == 'prop_of_ac':
            action = closest_action(1)
            return action
        else:
            raise Exception('Unknown Action Type')

        return action

# Define the Random Agent
class RandomAgent:
    def __init__(self, env):
        self.action_space = env.action_space

    def act(self, state):
        return np.random.choice(self.action_space)

# Define the Greedy Agent
class GreedyAgent:
    def __init__(self, env):
        self.action_space = env.action_space

    def act(self, state):
        return 1  # Always buy

# Evaluate the agents
def evaluate_agent(agent, env, num_episodes=10):
    episode_rewards = []
    for _ in range(num_episodes):
        obs = env.reset()
        episode_reward = 0
        while not env.done:
            action = agent.act(obs)
            obs, reward, done = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    return episode_rewards

# Load data (replace with actual data loading)
# Example: data = pd.read_csv('data.csv')
# Assuming data has columns 'PRICE', 'VOLUME', 'second', 'MID_PRICE'

# Initialize the environment
env = TradingEnv(data)

# Calculate parameters for the Almgren-Chriss agent
eta, rho, sigma, tau, lamb, kappa = env.calculate_parameters()

# Initialize agents
agents = [
    AlmgrenChrissAgent(env, len(data), eta, rho, sigma, tau, lamb, kappa),
    RandomAgent(env),
    GreedyAgent(env)
]

# Evaluate agents
agent_rewards = {}
for agent in agents:
    agent_name = agent.__class__.__name__
    rewards = evaluate_agent(agent, env)
    agent_rewards[agent_name] = rewards

# Print and compare results
for agent_name, rewards in agent_rewards.items():
    print(f"{agent_name}: Average Cumulative Reward = {np.mean(rewards)}")

# Plot cumulative rewards using Plotly
for agent_name, rewards in agent_rewards.items():
    fig = px.line(
        x=range(len(rewards)),
        y=np.cumsum(rewards),
        labels={'x': 'Episodes', 'y': 'Cumulative Reward'},
        title=f'Cumulative Rewards Comparison - {agent_name}'
    )
    fig.show()


/tmp/ipykernel_24/991563101.py:164: RuntimeWarning: overflow encountered in cosh
  nj = 2 * np.sinh(0.5 * self.kappa * self.tau) * np.cosh(self.kappa * (
/tmp/ipykernel_24/991563101.py:165: RuntimeWarning: overflow encountered in sinh
  self.time_horizon - (self.j - 0.5) * self.tau)) * (1 / inventory) / np.sinh(self.kappa
/tmp/ipykernel_24/991563101.py:164: RuntimeWarning: invalid value encountered in scalar divide
  nj = 2 * np.sinh(0.5 * self.kappa * self.tau) * np.cosh(self.kappa * (


AlmgrenChrissAgent: Average Cumulative Reward = 0.0
RandomAgent: Average Cumulative Reward = 72.30957098094154
GreedyAgent: Average Cumulative Reward = 1.2255812312651193


In [6]:
class Environment:
    def __init__(self, data):
        self.data = data
        self.ac_dict = self.create_action_dict()
        self.ac_type = 'vanilla6'  # or 'vanilla20' or 'prop_of_ac', depending on the strategy

    def create_action_dict(self):
        # Define action dictionary for two actions: buy and hold
        action_dict = {0: 'hold', 1: 'buy'}
        return action_dict

    def calculate_parameters(self):
        # Extract necessary columns
        prices = self.data['PRICE']
        volumes = self.data['VOLUME']
        
        # Calculate sigma (volatility)
        returns = prices.pct_change().dropna()
        sigma = returns.std()

        # Calculate tau (average time between trades in seconds)
        time_diffs = self.data['second'].diff().dropna()
        tau = time_diffs.mean()

        # Calculate eta and rho based on a more detailed market impact model
        eta = self.calculate_eta()
        rho = self.calculate_rho()

        # Set a default value for lambda (risk aversion parameter)
        lamb = 1e-6

        # Calculate kappa
        kappa = np.sqrt(abs(lamb * sigma**2 / (eta * (1 - rho * tau / (2 * eta)))))
        
        return eta, rho, sigma, tau, lamb, kappa

    def calculate_eta(self):
        # Estimate the market impact cost per unit traded
        price_impact = self.data['PRICE'] - self.data['MID_PRICE']
        volume = self.data['VOLUME']
        
        # Filter out zero volumes to avoid division by zero
        valid_indices = volume > 0
        price_impact = price_impact[valid_indices]
        volume = volume[valid_indices]

        # Calculate the market impact per unit volume traded
        market_impact_per_volume = price_impact / volume
        eta = market_impact_per_volume.mean()  # Average market impact per unit volume

        return eta

    def calculate_rho(self):
        # Calculate the standard deviation of returns as a proxy for risk
        returns = self.data['PRICE'].pct_change().dropna()
        return_std = returns.std()

        # Risk aversion might be proportional to return standard deviation
        rho = return_std * 1e-2  # Adjust the scaling factor as necessary

        return rho


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

class TradingEnv:
    def __init__(self, data, ac_dict=None, ac_type='vanilla6'):
        self.data = data
        self.current_step = 0
        self.done = False
        
        # Define action space: 0 for hold, 1 for buy
        self.action_space = [0, 1]
        
        # Define observation space: based on the number of columns in data
        self.observation_space = len(data.columns)
        
        # Initialize other variables specific to your environment
        self.inventory = 0  # Starting with zero inventory
        
        # Metrics tracking
        self.cumulative_reward = 0
        self.cumulative_rewards = []
        
        # Assign action dictionary and type
        self.ac_dict = ac_dict if ac_dict is not None else {0: 'hold', 1: 'buy'}
        self.ac_type = ac_type
    
    def reset(self):
        self.current_step = 0
        self.done = False
        self.inventory = 0
        self.cumulative_reward = 0
        self.cumulative_rewards = []
        return self._next_observation()
    
    def _next_observation(self):
        # Return current observation based on the data at current_step
        obs = self.data.iloc[self.current_step].values
        return obs
    
    def step(self, action):
        # Take a step in the environment based on the action
        current_row = self.data.iloc[self.current_step]
        self.current_step += 1
        
        reward = self._calculate_reward(current_row, action)
        self.cumulative_reward += reward
        self.cumulative_rewards.append(self.cumulative_reward)
        
        # Update inventory based on action
        if action == 1:  # Buy action
            self.inventory += 1
        
        # Check if episode is done
        if self.current_step >= len(self.data) - 1:
            self.done = True
        
        next_observation = self._next_observation()
        return next_observation, reward, self.done
    
    def _calculate_reward(self, row, action):
        # Calculate reward based on action and current state (row)
        if action == 1:  # Buy action
            reward = row['PRICE']  # Example: reward could be the current price
        else:  # Hold action
            reward = 0  # No reward for holding
        
        return reward
    
    def render(self):
        # Optional: Implement rendering if needed for visualization
        pass
    
    def plot_cumulative_rewards(self):
        fig = px.line(
            x=range(len(self.cumulative_rewards)),
            y=self.cumulative_rewards,
            labels={'x': 'Steps', 'y': 'Cumulative Reward'},
            title='Cumulative Reward over Episodes'
        )
        fig.show()


In [8]:
class AlmgrenChrissAgent:
    def __init__(self, env, time_horizon, eta, rho, sigma, tau, lamb, kappa):
        self.ac_dict = deepcopy(env.ac_dict)
        self.ac_type = deepcopy(env.ac_type)
        self.eta = eta
        self.rho = rho
        self.sigma = sigma
        self.tau = tau
        self.time_horizon = time_horizon
        self.steps, self.j = time_horizon / self.tau, 1
        self.kappa = kappa

    def reset(self):
        self.j = 1

    def act(self, state):
        def closest_action(nj):
            # Since there are only two actions, use a simple threshold
            action = 1 if nj >= 0.5 else 0
            return action

        if self.ac_type in ['vanilla6', 'vanilla20']:
            inventory = state[1]  # Adjust index if necessary
            if self.kappa == 0:
                nj = self.tau / self.time_horizon * (1 / inventory)
            else:
                nj = 2 * np.sinh(0.5 * self.kappa * self.tau) * np.cosh(self.kappa * (
                        self.time_horizon - (self.j - 0.5) * self.tau)) * (1 / inventory) / np.sinh(self.kappa
                                                                                                    * self.time_horizon)
            self.j += 1
            if self.j == self.steps + 1:
                nj = 1
            action = closest_action(nj)

            return action
        elif self.ac_type == 'prop_of_ac':
            action = closest_action(1)
            return action
        else:
            raise Exception('Unknown Action Type')

        return action


In [9]:
class RandomAgent:
    def __init__(self, env):
        self.env = env
    
    def act(self, obs):
        return np.random.choice(self.env.action_space)
    
class GreedyAgent:
    def __init__(self, env):
        self.env = env
    
    def act(self, obs):
        # Simple greedy agent: always buys if the price is lower than the previous step
        if self.env.current_step > 0:
            prev_price = self.env.data.iloc[self.env.current_step - 1]['PRICE']
            current_price = obs[0]
            if current_price < prev_price:
                return 1  # Buy
        return 0  # Hold

In [10]:
def evaluate_agent(agent, env, num_episodes=10):
    episode_rewards = []
    for _ in range(num_episodes):
        obs = env.reset()
        episode_reward = 0
        while not env.done:
            action = agent.act(obs)
            obs, reward, done = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    return episode_rewards

# Initialize agents
agents = [
    AlmgrenChrissAgent(env, len(data), eta, rho, sigma, tau, lamb, kappa),
    RandomAgent(env),  # Uncomment and define RandomAgent if needed
    GreedyAgent(env)  # Uncomment and define GreedyAgent if needed
]

# Evaluate agents
agent_rewards = {}
for agent in agents:
    agent_name = agent.__class__.__name__
    rewards = evaluate_agent(agent, env)
    agent_rewards[agent_name] = rewards

# Print and compare results
for agent_name, rewards in agent_rewards.items():
    print(f"{agent_name}: Average Cumulative Reward = {np.mean(rewards)}")

# Optionally, plot cumulative rewards using Plotly
for agent_name, rewards in agent_rewards.items():
    fig = px.line(
        x=range(len(rewards)),
        y=np.cumsum(rewards),
        labels={'x': 'Episodes', 'y': 'Cumulative Reward'},
        title=f'Cumulative Rewards Comparison - {agent_name}'
    )
    fig.show()


/tmp/ipykernel_24/3101799493.py:27: RuntimeWarning:

overflow encountered in cosh

/tmp/ipykernel_24/3101799493.py:28: RuntimeWarning:

overflow encountered in sinh

/tmp/ipykernel_24/3101799493.py:27: RuntimeWarning:

invalid value encountered in scalar divide



AlmgrenChrissAgent: Average Cumulative Reward = 0.0
RandomAgent: Average Cumulative Reward = 52.294941266454714
GreedyAgent: Average Cumulative Reward = 82716.30837655396
